# Reward Effectiveness Analysis

This notebook analyzes the effectiveness of different rewards across customer segments and campaigns.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime, timedelta

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## Load Data

In [ ]:
# Load rewards data
with open('../data/rewards.json', 'r') as f:
    rewards = json.load(f)
    
# Load events data (which contains reward claims)
with open('../data/events.json', 'r') as f:
    events = json.load(f)

# Load segmented customers
customers_df = pd.read_csv('../data/segmented_customers.csv')
    
# Convert to DataFrames
rewards_df = pd.DataFrame(rewards)
events_df = pd.DataFrame(events)

# Display basic info
print(f"Loaded {len(rewards_df)} rewards, {len(events_df)} events, and {len(customers_df)} customers")
rewards_df.head()

## Filter and Prepare Reward Claim Data

In [ ]:
# Filter events to get only reward claims
reward_claims = events_df[events_df['event_type'] == 'reward_claim'].copy()

# Extract reward ID from metadata
reward_claims['reward_id'] = reward_claims['metadata'].apply(lambda x: x.get('reward_id', None))

# Convert timestamp to datetime
reward_claims['timestamp'] = pd.to_datetime(reward_claims['timestamp'])

# Merge with rewards data
reward_claims = pd.merge(reward_claims, rewards_df, left_on='reward_id', right_on='id', how='left')

# Merge with customer data to get segments
reward_claims = pd.merge(reward_claims, customers_df[['id', 'segment']], left_on='customer_id', right_on='id', how='left', suffixes=('_reward', '_customer'))

reward_claims.head()

## Analyze Reward Effectiveness by Type

In [ ]:
# Count claims by reward type
reward_type_counts = reward_claims['type'].value_counts()

# Calculate claim distribution
reward_type_distribution = reward_type_counts / reward_type_counts.sum() * 100

# Plot claim distribution by reward type
plt.figure(figsize=(10, 6))
reward_type_distribution.plot(kind='bar', color='skyblue')
plt.title('Reward Claims by Type (%)')
plt.xlabel('Reward Type')
plt.ylabel('Percentage of Total Claims')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate claim rate for each reward
# For each reward, count total number of times it was shown vs. claimed
# In a real scenario, we would have impression data
# For this analysis, we'll estimate based on reward age

# Helper function to calculate days since reward creation
def days_since_creation(created_at):
    try:
        created_date = pd.to_datetime(created_at)
        return (datetime.now() - created_date).days
    except:
        return 30  # Default fallback

# Add days since creation
rewards_df['days_active'] = rewards_df['created_at'].apply(days_since_creation)

# Count claims per reward
reward_claim_counts = reward_claims['reward_id'].value_counts().reset_index()
reward_claim_counts.columns = ['reward_id', 'claim_count']

# Merge with rewards data
reward_effectiveness = pd.merge(rewards_df, reward_claim_counts, left_on='id', right_on='reward_id', how='left')
reward_effectiveness['claim_count'] = reward_effectiveness['claim_count'].fillna(0)

# Estimate impressions based on days active (simplified)
reward_effectiveness['estimated_impressions'] = reward_effectiveness['days_active'] * 5  # Assume 5 impressions per day

# Calculate claim rate
reward_effectiveness['claim_rate'] = reward_effectiveness['claim_count'] / reward_effectiveness['estimated_impressions']
reward_effectiveness['claim_rate'] = reward_effectiveness['claim_rate'].fillna(0)

# Display top rewards by claim rate
top_rewards = reward_effectiveness.sort_values('claim_rate', ascending=False).head(10)
print("Top 10 rewards by claim rate:")
top_rewards[['name', 'type', 'value', 'claim_count', 'claim_rate']]

## Analyze Reward Effectiveness by Customer Segment

In [ ]:
# Count claims by segment
segment_counts = reward_claims['segment'].value_counts()

# Count customers by segment for normalization
customer_segment_counts = customers_df['segment'].value_counts()

# Calculate claims per customer by segment
claims_per_customer = pd.DataFrame({
    'claims': segment_counts,
    'customers': customer_segment_counts
}).fillna(0)

claims_per_customer['claims_per_customer'] = claims_per_customer['claims'] / claims_per_customer['customers']

# Plot claims per customer by segment
plt.figure(figsize=(10, 6))
claims_per_customer['claims_per_customer'].sort_values(ascending=False).plot(kind='bar', color='skyblue')
plt.title('Average Reward Claims per Customer by Segment')
plt.xlabel('Segment')
plt.ylabel('Claims per Customer')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Create a cross-tabulation of segment vs. reward type
segment_reward_type = pd.crosstab(reward_claims['segment'], reward_claims['type'], normalize='index') * 100

# Plot heatmap of segment vs. reward type preferences
plt.figure(figsize=(12, 8))
sns.heatmap(segment_reward_type, annot=True, cmap='YlGnBu', fmt='.1f')
plt.title('Reward Type Preferences by Customer Segment (%)')
plt.ylabel('Customer Segment')
plt.xlabel('Reward Type')
plt.tight_layout()
plt.show()

## Analyze Reward Value and Conditions

In [ ]:
# Extract conditions from rewards
def has_min_purchase(conditions):
    try:
        return 'min_purchase' in conditions
    except:
        return False
    
def get_min_purchase(conditions):
    try:
        return conditions.get('min_purchase', 0)
    except:
        return 0

reward_effectiveness['has_min_purchase'] = reward_effectiveness['conditions'].apply(has_min_purchase)
reward_effectiveness['min_purchase'] = reward_effectiveness['conditions'].apply(get_min_purchase)

# Compare claim rates for rewards with and without minimum purchase
min_purchase_comparison = reward_effectiveness.groupby('has_min_purchase')['claim_rate'].mean() * 100

plt.figure(figsize=(8, 6))
min_purchase_comparison.plot(kind='bar', color=['skyblue', 'lightcoral'])
plt.title('Effect of Minimum Purchase Requirement on Claim Rate')
plt.xlabel('Has Minimum Purchase Requirement')
plt.ylabel('Average Claim Rate (%)')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze relationship between reward value and claim rate
plt.figure(figsize=(10, 6))
plt.scatter(reward_effectiveness['value'], reward_effectiveness['claim_rate'] * 100, 
            alpha=0.7, s=80, c=reward_effectiveness['days_active'], cmap='viridis')
plt.colorbar(label='Days Active')
plt.title('Relationship Between Reward Value and Claim Rate')
plt.xlabel('Reward Value')
plt.ylabel('Claim Rate (%)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Analyze Reward Performance by Segment and Type

In [ ]:
# Group claimed rewards by segment and type
segment_type_performance = reward_claims.groupby(['segment', 'type']).size().reset_index(name='claims')

# Pivot for visualization
segment_type_pivot = segment_type_performance.pivot(index='segment', columns='type', values='claims').fillna(0)

# Plot grouped bar chart
segment_type_pivot.plot(kind='bar', figsize=(12, 6))
plt.title('Reward Claims by Segment and Type')
plt.xlabel('Customer Segment')
plt.ylabel('Number of Claims')
plt.legend(title='Reward Type')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Optimal Reward Recommendations by Segment

In [ ]:
# Calculate effectiveness score for each segment-reward type combination
# Normalize by number of customers in each segment
segment_reward_effectiveness = segment_type_performance.copy()

# Merge with customer counts
segment_counts_df = pd.DataFrame({
    'segment': customer_segment_counts.index,
    'customer_count': customer_segment_counts.values
})

segment_reward_effectiveness = pd.merge(segment_reward_effectiveness, segment_counts_df, on='segment', how='left')

# Calculate claims per 100 customers
segment_reward_effectiveness['claims_per_100'] = segment_reward_effectiveness['claims'] / segment_reward_effectiveness['customer_count'] * 100

# Get top reward types for each segment
top_segment_rewards = segment_reward_effectiveness.sort_values(['segment', 'claims_per_100'], ascending=[True, False])
top_segment_rewards_pivot = top_segment_rewards.pivot(index='segment', columns='type', values='claims_per_100').fillna(0)

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(top_segment_rewards_pivot, annot=True, cmap='YlGnBu', fmt='.1f')
plt.title('Reward Effectiveness by Segment (Claims per 100 Customers)')
plt.ylabel('Customer Segment')
plt.xlabel('Reward Type')
plt.tight_layout()
plt.show()

In [ ]:
# Generate reward recommendations for each segment
segment_recommendations = {}

for segment in top_segment_rewards['segment'].unique():
    segment_data = top_segment_rewards[top_segment_rewards['segment'] == segment]
    top_types = segment_data.sort_values('claims_per_100', ascending=False).head(3)
    
    segment_recommendations[segment] = {
        'top_reward_types': top_types['type'].tolist(),
        'effectiveness_scores': top_types['claims_per_100'].tolist(),
        'recommended_value_range': [10, 25] if segment == 'VIP' else [5, 15],
        'min_purchase_recommended': segment not in ['At Risk', 'Recent']
    }

# Display recommendations
for segment, recommendations in segment_recommendations.items():
    print(f"\n{segment} Segment Recommendations:")
    print(f"Top Reward Types: {', '.join(recommendations['top_reward_types'])}")
    print(f"Effectiveness Scores: {[round(score, 2) for score in recommendations['effectiveness_scores']]}")
    print(f"Recommended Value Range: ${recommendations['recommended_value_range'][0]} - ${recommendations['recommended_value_range'][1]}")
    print(f"Include Minimum Purchase: {'Yes' if recommendations['min_purchase_recommended'] else 'No'}")

## Export Optimized Reward Configurations

In [ ]:
# Create optimized reward configurations
optimized_rewards = []

for segment, recommendations in segment_recommendations.items():
    for reward_type in recommendations['top_reward_types']:
        # Create reward for this segment and type
        value = np.random.uniform(recommendations['recommended_value_range'][0], 
                                   recommendations['recommended_value_range'][1])
        
        conditions = {}
        if recommendations['min_purchase_recommended']:
            conditions['min_purchase'] = 50 if segment == 'VIP' else 25
            
        reward = {
            "id": f"{segment.lower()}_{reward_type}_{len(optimized_rewards)}",
            "name": f"{reward_type.title()} for {segment}",
            "description": f"Optimized {reward_type} reward for {segment} customers",
            "type": reward_type,
            "value": round(value, 2),
            "target_segment": segment,
            "conditions": conditions,
            "created_at": datetime.now().isoformat()
        }
        
        optimized_rewards.append(reward)

# Save optimized rewards
with open('../data/optimized_rewards.json', 'w') as f:
    json.dump(optimized_rewards, f, indent=2)
    
print(f"Saved {len(optimized_rewards)} optimized reward configurations to '../data/optimized_rewards.json'")